# New York taxis trips

This homework is about New York taxi trips. Here is something from [Todd Schneider](https://toddwschneider.com/posts/analyzing-1-1-billion-nyc-taxi-and-uber-trips-with-a-vengeance/):

> The New York City Taxi & Limousine Commission has released a  detailed historical dataset covering over 1 billion individual taxi trips in the city from January 2009 through December 2019. 
Taken as a whole, the detailed trip-level data is more than just a vast list of taxi pickup and drop off coordinates: it's a story of a City. 
How bad is the rush hour traffic from Midtown to JFK? 
Where does the Bridge and Tunnel crowd hang out on Saturday nights?
What time do investment bankers get to work? How has Uber changed the landscape for taxis?
The dataset addresses all of these questions and many more.

The NY taxi trips dataset has been plowed by series of distinguished data scientists.
The dataset is available from on Amazon S3 (Amazon's cloud storage service).
The link for each file has the following form:

    https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_{year}-{month}.csv

There is one CSV file for each NY taxi service (`yellow`, `green`, `fhv`) and each calendar month (replacing `{year}` and `{month}` by the desired ones).
Each file is moderately large, a few gigabytes. 
The full dataset is relatively large if it has to be handled on a laptop (several hundred gigabytes).

You will focus on the `yellow` taxi service and a pair of months, from year 2015 and from year 2018. 
Between those two years, for hire vehicles services have taken off and carved a huge marketshare.

Whatever the framework you use, `CSV` files prove hard to handle. 
After downloading the appropriate files (this takes time, but this is routine), a first step will consist in converting the csv files into a more Spark friendly format such as `parquet`.

Saving into one of those formats require decisions about bucketing, partitioning and so on. Such decisions influence performance. It is your call.
Many people have been working on this dataset, to cite but a few:


- [1 billion trips with a vengeance](https://toddwschneider.com/posts/analyzing-1-1-billion-nyc-taxi-and-uber-trips-with-a-vengeance/)
- [1 billion trips with R and SQL ](http://freerangestats.info/blog/2019/12/22/nyc-taxis-sql)
- [1 billion trips with redshift](https://tech.marksblogg.com/billion-nyc-taxi-rides-redshift.html)
- [nyc-taxi](https://github.com/fmaletski/nyc-taxi-map)

Depending on your internet connection, **download the files** corresponding to **"yellow" taxis** for the years 2015 and 2018. Download **at least one month** (the same) for 2015 and 2018, if you can download all of them.

**Hint.** The 12 csv for 2015 are about 23GB in total, but the corresponding parquet file, if you can create it for all 12 months, is only about 3GB.

You **might** need the following stuff in order to work with GPS coordinates and to plot things easily.

In [ ]:
!pip install geojson geopandas plotly geopy

In [ ]:
!pip install ipyleaflet

For this homework **we will let you decide on the tools to use** (expected for Spark) and to **find out information all by yourself** (but don't hesitate to ask questions on the `slack` channel).

# Loading data as parquet files

We want to organize the data on a per year and per service basis. 
We want to end up with one `parquet` file for each year and each taxi service, since parquet is much better than CSV files.

**Hint.** Depending on your internet connection and your laptop, you can use only the "yellow" service and use one month of 2015 and 2018

CSV files can contain corrupted lines. You may have to work in order to perform ETL (Extract-Transform-Load) in order obtain a properly typed data frame.

You are invited to proceed as follows:

1. Try to read the CSV file without imposing a schema. 
1. Inspect the inferred schema. Do you agree with Spark's typing decision?
1. Eventually correct the schema and read again the data
1. Save the data into parquet files
1. In the rest of your work, **you will only use the parquet files you created**, not the csv files (don't forget to choose a partitioning column and a number of partitions when creating the parquet files).

**Hint.** Don't forget to ask `Spark` to use all the memory and ressources from your computer.

**Hint.** Don't foreget that you should specify a partitioning column and a number of partitions when creating the parquet files.

**Hint.** Note that the schemas of the 2015 and 2018 data are different...

**Hint.** When working on this, ask you and answer to the following questions:

1. What is the `StorageLevel` of the dataframe after reading the csv files?
1. What is the number of partitions of the dataframe? 
1. Is it possible to tune this number at loading time? 
1. Why would we want to modify the number of partitions when creating the parquet files?

In [ ]:
# import the usual suspects
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import sys
import timeit

%matplotlib inline
import seaborn as sns

sns.set_context("notebook", font_scale=1.2)

In [ ]:
# spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
from pyspark.sql.catalog import Catalog
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType

In [ ]:
#Start the SparkSession

conf = SparkConf().setAppName("Spark SQL Course")
conf.set("spark.executor.memory", "5g")
conf.set("spark.driver.memory", "5g")
conf.set("spark.cores.max", "4")
conf.set("spark.executor.cores", "4")
sc = SparkContext(conf=conf)

spark = (SparkSession
    .builder
    .appName("homework 3")
    .getOrCreate()
)


In [ ]:
sc.getConf().getAll()

In [ ]:
df_15_03 = spark.read\
             .format('csv')\
             .option("header", "true")\
             .option("mode", "FAILFAST")\
             .option("inferSchema", "true")\
             .option("sep", ",")\
             .load("nyc_taxi/yellow_tripdata_2015-03.csv")

df_15_03.printSchema()

In [ ]:
df_18_03 = spark.read\
             .format('csv')\
             .option("header", "true")\
             .option("mode", "FAILFAST")\
             .option("inferSchema", "true")\
             .option("sep", ",")\
             .load("nyc_taxi/yellow_tripdata_2018-03.csv")

df_18_03.printSchema()

In [ ]:
df_15_03.groupBy('store_and_fwd_flag').agg(fn.count(col("VendorID"))).show(20)


In [ ]:
#df_15_03.groupBy('payment_type').agg(fn.count(col("VendorID"))).show(20)

df_18_03.show(5)
#df_18_03.select('PULocationID').show(5)

In [ ]:
#We correct the schema:

from pyspark.sql.types import *

schema_15 = StructType([
    StructField("VendorID", IntegerType(), True),
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("pickup_longitude", DoubleType(), True),
    StructField("pickup_latitude", DoubleType(), True),
    StructField("RateCodeID", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("dropoff_longitude", DoubleType(), True),
    StructField("dropoff_latitude", DoubleType(), True),
    StructField("payment_type", IntegerType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)
])

schema_18 = StructType([
    StructField("VendorID", IntegerType(), True),
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("RateCodeID", IntegerType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("PULocationID", IntegerType(), True),
    StructField("DOLocationID", IntegerType(), True),
    StructField("payment_type", IntegerType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True)
])


In [ ]:
df_15_03 = spark.read\
             .format('csv')\
             .option("header", "true")\
             .option("mode", "FAILFAST")\
             .option("sep", ",")\
             .schema(schema_15)\
             .load("nyc_taxi/yellow_tripdata_2015-03.csv")

df_15_03.printSchema()



df_18_03 = spark.read\
             .format('csv')\
             .option("header", "true")\
             .option("mode", "FAILFAST")\
             .option("sep", ",")\
             .schema(schema_18)\
             .load("nyc_taxi/yellow_tripdata_2018-03.csv")

df_18_03.printSchema()

In [ ]:
#number of partitions of the dataframe:

print(df_15_03.rdd.getNumPartitions())

print(df_18_03.rdd.getNumPartitions())

#k = sc.parallelize(df_15_03)

In [ ]:

df_15_03.write.format("parquet").mode("overwrite").save('data/yellow_tripdata_2015-03.parquet')
df_18_03.write.format("parquet").mode("overwrite").save('data/yellow_tripdata_2018-03.parquet')


# Investigate (at least) one month of data in 2015

From now on, you will be using **the parquet files you created for 2015**.

We shall visualize several features of taxi traffic during one calendar month
in 2015 and the same calendar month in 2018.

**Hint.** In order to build appealing graphics, you may stick to `matplotlib + seaborn`, you can use also
`plotly`, which is used a lot to build interactive graphics, but you can use whatever you want.

The following longitudes and lattitudes encompass Newark and JFK airports, Northern Manhattan and Verazzano bridge.

In [ ]:

df_mar15 = spark.read.parquet('data/yellow_tripdata_2015-03.parquet')

df_mar15 = df_mar15.filter(col("tpep_pickup_datetime") >= "2015-01-01 00:00")\
                .filter(col("tpep_pickup_datetime") < "2016-01-01 00:00")
df_mar15.count()


In [ ]:
df_mar18 = spark.read.parquet('data/yellow_tripdata_2018-03.parquet')

df_mar18 = df_mar18.filter(col("tpep_pickup_datetime") >= "2018-01-01 00:00")\
                .filter(col("tpep_pickup_datetime") < "2019-01-01 00:00")

In [ ]:
long_min = -74.10
long_max = -73.70
lat_min = 40.58
lat_max = 40.90

start = time.time()


mar15_filtred = df_mar15.where(col('pickup_longitude').between(long_min, long_max))\
                    .where(col('pickup_latitude').between(lat_min, lat_max))\
                    .where(col('dropoff_longitude').between(long_min, long_max))\
                    .where(col('dropoff_latitude').between(lat_min, lat_max))

end = time.time()
print(end - start)

1. Using these boundaries, **filter the 2015 data** (using pickup and dropoff longitude and latitude) and count the number of trips for each value of `passenger_count` and make a plot of that.

In [ ]:
mar15_psgr = mar15_filtred.groupBy(col('passenger_count')).count()

        
mar15_psgr.show()

sns.set()

g = sns.lineplot(x = "passenger_count",
                 y = "count",
                 data = mar15_psgr.toPandas()).set_title("Number of trips for each value of passenger_count")

plt.show()

Trips with $0$ or larger than $7$ passengers are pretty rare.
We suspect these to be outliers. 
We need to explore these trips further in order to understand what might be wrong
with them

1. What's special with trips with zero passengers?

It's the time it takes for the driver to find/begin a new ride.

1. What's special with trips with more than $6$ passengers?



1. What is the largest distance travelled during this month? Is it the first taxi on the moon?

1.5420061E7

1. Plot the distribution of the `trip_distance` (using an histogram for instance) during year 2015. Focus on trips with non-zero trip distance and trip distance less than 30 miles.

In [ ]:
mar15_0_psgr = mar15_filtred.where(col('passenger_count')==0)\
                            .orderBy(mar15_filtred.trip_distance.desc())
#mar15_0_psgr.show(5)

#largest distance with 0 passengers: 24.1


mar15_6_psgr = mar15_filtred.where(col('passenger_count')>=6)\
                            
#mar15_6_psgr.show(5)

#largest distance with more than 6 passengers: 61.75

largest_dist_15 = mar15_filtred.orderBy(mar15_filtred.trip_distance.desc())\
                    .limit(1)
#largest_dist_15.show()

nb_0dist_6 =mar15_filtred.where(col('passenger_count')>=6).where(col('trip_distance')==0).count()
nb_0dist_6

In [ ]:
mar15_trip_dist = mar15_filtred.where(col('trip_distance').between(0, 30))\
                                .groupBy(col('trip_distance'))\
                                .count()

sns.set()

g = sns.lineplot(x = "trip_distance",
                 y = "count",
                 data = mar15_trip_dist.toPandas()).set_title("Distribution of the trip_distance during year 2015")

plt.show()

#ax = mar15_trip_dist.toPandas().plot.hist(by='trip_distance',bins=15).set_title("Distribution of the trip_distance during year 2015")


Let's look at what Spark does for these computations

1. Use the `explain` method or have a look at the [Spark UI](http://localhost:4040/SQL/) to analyze the job. You should be able to assess 
    - Parsed Logical Plan
    - Analyzed Logical Plan
    - Optimized Logical Plan
    - Physical Plan
1. Do the Analyzed Logical Plan and Optimized Logical Plan differ? Spot the differences if any. How would a RDBMS proceed with such a query?

The Analyzed Logical Plan and Optimized Logical Plan differ with teh type of the elments. Indeed, in the Analyzed Logical Plan, there is all the columns of the dataframe with their types before the rest of the the logical plan. In the optimized logical plan, Spark does optimization itself.

1. How does the physical plan differ from the Optimized Logical Plan? What are the keywords you would not expect in a RDBMS? What is their meaning? 

A logical plan is a tree that represents both schema and data. These trees are manipulated and optimized by catalyst framework. The optimized logical plan is converted to a physical plan for further execution. These plans lie inside the DataFrame API. Spark will check up multiple physical plans and decide the best optimal physical plan. And finally the Best Physical Plan runs in our cluster. Physical Plan is responsible for deciding the type of join, sequence of the execution of filter, where, groupBy clause, etc.

1. Inspect the stages on [Spark UI](http://localhost:4040/stages/stage). How many *stages* are necessary to complete the Spark job? What are the roles of `HashAggregate` and `Exchange hashpartitioning`?

29

HashAggregateExec is a unary physical operator for hash-based aggregation that is created indirectly when Aggregation execution planning strategy selects the aggregate physical operator for an Aggregate logical operator.

1. Does the physical plan perform `shuffle` operations? If yes how many?
5 shuffle write and 7 shuffle read
1. What are tasks with respect to stages (in Spark language)? How many tasks are your stages made of?

In [ ]:
df_mar15.explain(extended = True)
#df_mar18.explain(extended = True)

#mar15_filtred.explain(extended = True)

Now, compute the following and produce relevant plots:

1. Break down the trip distance distribution for each day of week
1. Count the number of distinct pickup location
1. Compute and display tips and profits as a function of the pickup location

In [ ]:
#Trip distance distribution for each day of week :

from pyspark.sql.functions import dayofweek
from pyspark.sql.functions import date_format
from pyspark.sql.functions import col

mar15_days = df_mar15.withColumn("week_day", date_format(col("tpep_pickup_datetime"), "EEEE"))\
                        .where(col('trip_distance').between(0, 30))\
                        .groupBy(col('trip_distance'),col('week_day'))\
                        .count()

mar15_days = mar15_days.orderBy(mar15_days.week_day.desc())
#mar15_days.show(20)

sns.set()

g = sns.FacetGrid(mar15_days.toPandas(), col="week_day", col_wrap=5)
g = g.map(sns.lineplot, "trip_distance", "count")

g.fig.suptitle("Distribution of the trip_distance during year 2015, by day of week", y=1.05)

plt.show()

In [ ]:
#The number of distinct pickup location :

import pyspark.sql.functions as func
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import col
from pyspark.sql.functions import udf


def concat_str(a,b):
  return str(a) + str(b)
spark.udf.register("concat_str", concat_str)
first_udf = udf(concat_str)

df = df_mar15.select("pickup_longitude","pickup_latitude", first_udf("pickup_longitude","pickup_latitude").alias("tuple"))

gr = df.agg(countDistinct('tuple'))
gr.show()

#9934537

In [ ]:
#Tips as a function of the pickup location :

import pyspark.sql.functions as func
from pyspark.sql.functions import col
from matplotlib import pyplot


df_location = mar15_filtred.where(col('tip_amount').between(0,100))\
                    .withColumn('pickup_longitude',func.round(mar15_filtred['pickup_longitude'],3))\
                    .withColumn('pickup_latitude',func.round(mar15_filtred['pickup_latitude'],3))\
                    .groupBy(col('pickup_longitude'),col('pickup_latitude'),col('tip_amount')).count()

df_location.show(4)


fig, ax = plt.subplots(figsize=(11.7, 8.27))

sns.set()


k = sns.scatterplot(x='pickup_longitude',
                    y='pickup_latitude',
                    hue='tip_amount',
                    size='tip_amount',
                    data=df_location.toPandas(),
                   ax=ax).set_title("Tips as a function of the pickup location during year 2015")

plt.show()


In [ ]:
#Profits as a function of the pickup location:

import pyspark.sql.functions as func
from pyspark.sql.functions import col
from matplotlib import pyplot


df_tupless = mar15_filtred.withColumn("profits", col("total_amount")-(col("mta_tax")+col("tolls_amount")))\
                    .where(col('profits').between(-20,100))\
                    .withColumn('pickup_longitude',func.round(mar15_filtred['pickup_longitude'],3))\
                    .withColumn('pickup_latitude',func.round(mar15_filtred['pickup_latitude'],3))\
                    .groupBy(col('pickup_longitude'),col('pickup_latitude'),col('profits')).count()

df_tupless.show(4)


fig, ax = plt.subplots(figsize=(11.7, 8.27))

sns.set()


k = sns.scatterplot(x='pickup_longitude',
                    y='pickup_latitude',
                    hue='profits',
                    size='profits',
                    data=df_tupless.toPandas(),
                   ax=ax).set_title("Profits as a function of the pickup location during year 2015")

plt.show()


# Investigate one month of trips data in 2015 and 2018

 Consider one month of trips data from `yellow` taxis for each year

1. Filter and cache/persist the result

## Assessing seasonalities and looking at time series

Compute and plot the following time series indexed by day of the week and hour of day:

1. The number of pickups
1. The average fare
1. The average trip duration
1. Plot the average number of ongoing trips



In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from pyspark.sql.functions import date_format
from shapely.geometry import Polygon


In [ ]:
def group_weekday(df): 
    return df.select('fare_amount',
                          (fn.unix_timestamp('tpep_dropoff_datetime')- fn.unix_timestamp('tpep_pickup_datetime')).alias('duration'), 
                          date_format('tpep_pickup_datetime', 'E').alias('week_day'), 
                          date_format('tpep_pickup_datetime', 'HH').cast(IntegerType()).alias('hour'),
                          date_format('tpep_pickup_datetime', 'YYYY').alias('year'))
                            

start = time.time()

df_grp = group_weekday(df_mar18).union(group_weekday(mar15_filtred))

df_grp = df_grp.groupBy(col('hour'), col('week_day'), col('year')).agg(fn.round(fn.mean('duration'),1).alias('average_duration'),
                                                                   fn.round(fn.mean('fare_amount'),2).alias('average_fare'),
                                                                   fn.count('fare_amount').alias('nb_trips'))
df_grp.orderBy('year').show(10)

end = time.time()
print(end - start)

In [ ]:
pd_grp = df_grp.orderBy('hour').toPandas()

#pd_grp = pd_grp.drop(columns=['year']).pivot(index='hour', columns='week_day', values=['average_duration', 'nb_trips', 'average_fare'])

In [ ]:
def plot_by_hour(data, column, ytitle):
    fig = px.line(data, x="hour", y=column, color="week_day",
                  facet_col="year", hover_name="year")

    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.update_layout(
        autosize=False,
        width=900,
        height=600
    )

    fig.update_yaxes(title_text=ytitle)
    fig.update_xaxes(showticklabels=True)
    fig.show()

In [ ]:
plot_by_hour(pd_grp, "average_duration", 'Average duration')

In [ ]:
plot_by_hour(pd_grp, "average_fare", 'Average fare')

In [ ]:
plot_by_hour(pd_grp, "nb_trips", 'Total trips')

## Rides to the airports

In order to find the longitude and lattitude of JFK and Newark airport as well as the longitude and magnitudes 
of Manhattan, you can use a service like [geojson.io](http://geojson.io/).
Plot the following time series, indexed the day of the week and hour of the day

1. Median duration of taxi trip leaving Midtown (Southern Manhattan) headed for JFK Airport
1. Median taxi duration of trip leaving from JFK Airport to Midtown (Southern Manhattan)

In [ ]:
manhattan = Polygon([(-73.971, 40.727),
                     (-74.013, 40.742),
                     (-73.996, 40.773),
                     (-73.958, 40.758),
                     (-73.971, 40.727)])

jfk = Polygon([(-73.82,40.659),
               (-73.819,40.648),
               (-73.769,40.626),
               (-73.754,40.646),
               (-73.79,40.665),
               (-73.82,40.659)])

In [ ]:
start = time.time()
df_mar15_geo = df_mar15.limit(1000000).select((fn.unix_timestamp('tpep_dropoff_datetime')- fn.unix_timestamp('tpep_pickup_datetime')).alias('duration'),
                'pickup_longitude',
                'pickup_latitude',
                'dropoff_longitude',
                'dropoff_latitude')
mid = time.time()

df_geo = df_mar15_geo.toPandas()

end = time.time()
print(mid-start, end-mid)
df_geo

In [ ]:
gdf_pu = geopandas.GeoDataFrame(df_geo.copy(), 
                             geometry=geopandas.points_from_xy(df_geo.pickup_longitude, df_geo.pickup_latitude))

gdf_do = geopandas.GeoDataFrame(df_geo.copy(), 
                             geometry=geopandas.points_from_xy(df_geo.dropoff_longitude, df_geo.dropoff_latitude))

In [ ]:
pu_ma_to_jfk = gdf_pu[gdf_pu.geometry.within(manhattan) & gdf_do.geometry.within(jfk)]

pu_ma_to_jfk['duration'].median()

In [ ]:
pu_jfkto_ma = gdf_pu[gdf_pu.geometry.within(jfk) & gdf_do.geometry.within(manhattan)]

pu_jfkto_ma['duration'].median()



## Geographic information

For this, you will need to find tools to display maps and to build choropeth maps.
We let you look and find relevant tools to do this.

1. Build a heatmap where color is a function of
    1. number of `pickups`
    2. number of `dropoffs`
    3. number of `pickups` with dropoff at some airport (JFK, LaGuardia, Newark)
2. Build a choropeth map where color is a function of
    1. number of pickups in the area
    1. ratio of number of payments by card/number of cash payments for pickups in the area
    2. ratio of total fare/trip duration for dropoff in the area
3. Build an interactive chorophet with a slider allowing the user to select an `hour of day` and where the color is a function of
    1. average number of dropoffs in the area during that hour the day
    2. average ratio of tip over total fare amount for pickups in the area at given hour of the day